In [45]:
from src.pdfinsight import pdf_extractor
import pandas as pd

In [ ]:
df = pdf_extractor("220 Performance Report (Construction).pdf", toc_pages = 1, )
df

In [48]:
# we check the ymin difference between each row INSTEAD OF BLOCK. 
# this is important in determining the para_thres
# text below the para_thres would be considered to be in the same block of text (ie para) 
# seems like if those between 0 to 20 is above 70%, we can set that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['ymin_round'].diff()], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.019157
(-0.01, 10.0]       0.452107
(10.0, 20.0]        0.325670
(20.0, 30.0]        0.126437
(30.0, 40.0]        0.026820
(40.0, 50.0]        0.019157
(50.0, 1000.0]      0.030651
Name: proportion, dtype: float64

In [49]:
# double check the ymin_thres between REFINED_BLOCKS
# as long as our defined para_thres of 20 is the majority
# then should be safe to use that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['block_ymin_diff']], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.093385
(-0.01, 10.0]       0.000000
(10.0, 20.0]        0.431907
(20.0, 30.0]        0.276265
(30.0, 40.0]        0.112840
(40.0, 50.0]        0.027237
(50.0, 1000.0]      0.058366
Name: proportion, dtype: float64